# Loading the credentials

In [ ]:
# Load the Twitter API credentials from the config file
import json
with open('config.json', 'r') as f:
    config = json.load(f)
    consumer_key = config['consumer_key']
    consumer_secret = config['consumer_secret']
    access_token = config['access_token']
    access_token_secret = config['access_token_secret']

# Downloading twitter data

In [ ]:
import tweepy
from textblob import TextBlob
import pandas as pd
import os

# Authenticate with the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Define the topic you want to analyze
topic = 'volunteering'

# Check if the file already exists
if os.path.isfile('tweet_data.csv'):
    # Load the existing data from the CSV file
    tweet_data = pd.read_csv('tweet_data.csv')
    # Get the ID of the most recent tweet in the existing data
    since_id = tweet_data['id'].max()
else:
    # Create an empty DataFrame to store the tweet data
    tweet_data = pd.DataFrame(columns=['id', 'tweet', 'polarity', 'subjectivity', 'date'])
    # Set since_id to None to collect all tweets on the topic
    since_id = None

# Collect tweets on the topic for the last year (or since the most recent tweet in the existing data)
tweets = tweepy.Cursor(api.search_tweets, q=topic, lang='en', tweet_mode='extended', since_id=since_id).items()

# Loop through the tweets and perform sentiment analysis on each one
for tweet in tweets:
    tweet_id = tweet.id
    tweet_text = tweet.full_text
    tweet_date = tweet.created_at
    analysis = TextBlob(tweet_text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    tweet_data = tweet_data.append({'id': tweet_id, 'tweet': tweet_text, 'polarity': polarity, 'subjectivity': subjectivity, 'date': tweet_date}, ignore_index=True)

# Save the tweet data to a CSV file
tweet_data.to_csv('tweet_data.csv', index=False)

# Dashboard 

In [ ]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

# Load the tweet data
tweet_data = pd.read_csv('tweet_data.csv')

# Create a Plotly line chart of the sentiment over time
fig = px.line(tweet_data, x='date', y='polarity', title='Sentiment over Time')

# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    html.H1(children='Sentiment Analysis Dashboard'),
    html.Div(children='''A dashboard to display sentiment analysis on a specific topic.'''),
    dcc.Graph(id='sentiment-chart', figure=fig)
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)